In [3]:
import time
import qrcode
from projectq import MainEngine
from projectq.ops import H, CNOT, Measure, X, All
from projectq.backends import ClassicalSimulator, ResourceCounter
from projectq.meta import Compute, Uncompute
from pyzbar.pyzbar import decode
from PIL import Image

# Function to estimate error rate (assuming 0.1% error per gate)
def estimate_error_rate(total_gates):
    error_rate_per_gate = 0.001
    return error_rate_per_gate * total_gates * 100  # In percentage

# QR code generation and display
def generate_qr_code(data, title):
    qr = qrcode.make(data)
    qr.save(f"{title}.png")
    qr.show(title=title)
    return f"{title}.png"

# Read QR code and decode
def read_qr_code(filename):
    with Image.open(filename) as img:
        decoded_data = decode(img)[0].data.decode("utf-8")
    return decoded_data

# Double AES function using AES-64 for each layer
def Double_AES(eng, resource_check):
    # Generate QR code for plaintext and read back
    plaintext_data = "Sample plaintext message for AES-64"
    plaintext_qr_filename = generate_qr_code(plaintext_data, "Plaintext QR Code")
    plaintext_from_qr = read_qr_code(plaintext_qr_filename)
    print("Text from Plaintext QR Code:", plaintext_from_qr)

    # First AES Layer (AES-64)
    print("Running First AES Layer (AES-64)")
    x0, x1 = AES_64(eng, resource_check)

    # Convert intermediate ciphertext to QR code and read back
    first_layer_ciphertext = "First layer ciphertext representation"  # Placeholder for state
    first_layer_ciphertext_qr_filename = generate_qr_code(first_layer_ciphertext, "First Layer Ciphertext QR Code")
    first_layer_ciphertext_from_qr = read_qr_code(first_layer_ciphertext_qr_filename)
    print("Ciphertext from First Layer QR Code:", first_layer_ciphertext_from_qr)

    # Second AES Layer (AES-64), using output from first AES layer
    print("\nRunning Second AES Layer (AES-64)")
    AES_64(eng, resource_check, (x0, x1))

    # Convert final ciphertext to QR code and read back
    final_ciphertext = "Final ciphertext representation"  # Placeholder for state
    final_ciphertext_qr_filename = generate_qr_code(final_ciphertext, "Final Ciphertext QR Code")
    final_ciphertext_from_qr = read_qr_code(final_ciphertext_qr_filename)
    print("Ciphertext from Final QR Code:", final_ciphertext_from_qr)

    # Final encrypted text (ciphertext after both layers)
    print("\nFinal Ciphertext:")
    print("Block 1:", end=" ")
    print_state(eng, x1, 8)
    print("Block 2:", end=" ")
    print_state(eng, x0, 8)

    return x0, x1

# AES-64 function
def AES_64(eng, resource_check, input_state=None):
    if input_state is None:
        x0 = eng.allocate_qureg(32)
        x1 = eng.allocate_qureg(32)
    else:
        x0, x1 = input_state

    k = eng.allocate_qureg(64)  # 64-bit key for AES-64

    # Initial XOR with round constants for state and key
    if resource_check != 1:
        Round_constant_XOR(eng, x1, 0x12345678, 32)
        Round_constant_XOR(eng, x0, 0x12345678, 32)
        Round_constant_XOR(eng, k, 0x1234567812345678, 64)

    # Optional: Print plaintext and key
    if resource_check != 1:
        print('Plaintext\n')
        print_state(eng, x1, 8)
        print_state(eng, x0, 8)
        print('Key\n')
        print_state(eng, k, 8)

    # Perform encryption rounds (6 rounds for AES-64)
    for i in range(6):
        Keyshedule_64(eng, k, i, resource_check)
        SBox_bp12_all(eng, x0, x1, resource_check)
        x0, x1 = Shiftrow(eng, x0, x1)
        if i != 5:
            x0 = Maxi_mc(eng, x0)
            x1 = Maxi_mc(eng, x1)
        AddRoundkey(eng, x0, x1, k)

    return x0, x1

# Helper functions
def Keyshedule_64(eng, k, round, resource_check):
    pass

def SBox_bp12_all(eng, x0, x1, resource_check):
    pass

def Shiftrow(eng, x0, x1):
    return x1, x0

def Maxi_mc(eng, x):
    return x

def AddRoundkey(eng, x0, x1, k):
    CNOT32(eng, k[0:32], x0)
    CNOT32(eng, k[32:64], x1)

def Round_constant_XOR(eng, x, constant, bits):
    for i in range(bits):
        if (constant >> i) & 1:
            X | x[i]

def CNOT32(eng, a, b):
    for i in range(32):
        CNOT | (a[i], b[i])

def print_state(eng, b, n):
    All(Measure) | b
    print('0x', end='')
    print_hex(eng, b, n)
    print('\n')

def print_hex(eng, qubits, n):
    for i in reversed(range(n)):
        temp = 0
        temp += int(qubits[4 * i + 3]) * 8
        temp += int(qubits[4 * i + 2]) * 4
        temp += int(qubits[4 * i + 1]) * 2
        temp += int(qubits[4 * i])
        print(hex(temp).replace("0x", ""), end='')

# Main code to run the encryption and estimate cost
if __name__ == "__main__":
    # Initialize the simulator
    Simulate = ClassicalSimulator()
    eng = MainEngine(Simulate)

    # Measure the total execution time
    start_time = time.time()

    # Run Double AES for encryption
    Double_AES(eng, 0)
    encryption_time = time.time() - start_time

    # Estimate cost
    print('Estimate cost...')
    Resource = ResourceCounter()
    eng = MainEngine(Resource)
    Double_AES(eng, 1)

    # Calculate total gates and error rate
    total_gates = sum(Resource.gate_counts[key] for key in Resource.gate_counts if 'Gate' in str(key[0]))
    error_rate = estimate_error_rate(total_gates)

    # Print results
    print(f"Total execution time: {encryption_time:.4f} seconds")
    print(f"Estimated Error Rate: {error_rate:.4f}%")
    print(Resource)
    eng.flush()


Text from Plaintext QR Code: Sample plaintext message for AES-64
Running First AES Layer (AES-64)
Plaintext

0x12345678

0x12345678

Key

0x12345678

Ciphertext from First Layer QR Code: First layer ciphertext representation

Running Second AES Layer (AES-64)
Plaintext

0x00000000

0x00000000

Key

0x12345678

Ciphertext from Final QR Code: Final ciphertext representation

Final Ciphertext:
Block 1: 0x00000000

Block 2: 0x00000000

Estimate cost...
Text from Plaintext QR Code: Sample plaintext message for AES-64
Running First AES Layer (AES-64)
Ciphertext from First Layer QR Code: First layer ciphertext representation

Running Second AES Layer (AES-64)
Ciphertext from Final QR Code: Final ciphertext representation

Final Ciphertext:
Block 1: 0x00000000

Block 2: 0x00000000

Total execution time: 10.4252 seconds
Estimated Error Rate: 0.0000%
Gate class counts:
    AllocateQubitGate : 192
    CXGate : 768
    DeallocateQubitGate : 192
    MeasureGate : 64

Gate counts:
    Allocate : 192